## Data Prep Mortality rate, Human Development Index and Regions/Super_Regions

In [1]:
import numpy as np
import pandas as pd

from os import path


import warnings
warnings.filterwarnings('ignore')

In [2]:
p = path.join('Data', 'IHME-GBD_2016_Lung.csv')
p = path.abspath(p)

In [3]:
#Reading file
'''p2 = path.join("/Users/user/Documents/publication/Bioinformatics/LC/LC-Pub-IHME-GBD_2016/","IHME-GBD_2016_Lung_01.csv")
p2 = path.abspath(p2)
p2
df=pd.read_csv(p2)'''

'p2 = path.join("/Users/user/Documents/publication/Bioinformatics/LC/LC-Pub-IHME-GBD_2016/","IHME-GBD_2016_Lung_01.csv")\np2 = path.abspath(p2)\np2\ndf=pd.read_csv(p2)'

In [4]:
# Data are collected from: Global Health Data Exchange
LC_url = "Data/IHME-GBD_2016_Lung.csv"
LC_df = pd.read_csv(LC_url)

#Data are collected from: UNITED NATIONS DEVELOPMENT PROGRAMME, Human Development Reports

HDI_url = "Data/Human development index (HDI)1990-2015.csv"
HDI_df = pd.read_csv(HDI_url)


r_sr_url = "Data/r_rs.csv"
r_sr_df = pd.read_csv(r_sr_url)
r_sr_df.drop("Unnamed: 0", axis=1,inplace=True)

### Working with Human Developement Index File: 
> #### Need to rename year columns to be able to change the current wide form into long form   

In [5]:
for i in range(2,HDI_df.columns[2:].size+2):
    HDI_df.rename(columns={HDI_df.columns[i] : "HDI-"+str(HDI_df.columns[i])}, inplace=True)

In [6]:
#Add id to each country
HDI_df["id"]= HDI_df.index

In [7]:
#Wide to Long form
HDI_df_long = pd.wide_to_long(HDI_df,"HDI", i= ["Country"] ,j = "Year", sep="-", suffix='\w')

In [8]:
#Sort and Reset index
HDI_df_long.sort_index(inplace=True)
HDI_df_long.reset_index(inplace=True)

In [9]:
#Select needed columns
HDI_df_long=HDI_df_long[["Country","id","Year","HDI"]]

In [10]:
# Rename columns
HDI_df_long.rename(columns={"Country":"country","Year" : "year"},inplace=True)

In [11]:
HDI_df_long.columns

Index(['country', 'id', 'year', 'HDI'], dtype='object')

### Working with Lung Cancer File: 

In [12]:
#Select needed columns
LC_df = LC_df[["location","year","sex","val"]]

In [13]:
# Rename columns
LC_df.rename(columns={"location": "country","val": "MR"} , inplace=True)

> ### Need to add region and super region 

In [14]:
LC_df["region"] =""
LC_df["super_region"]=""

In [15]:
print(r_sr_df.columns)
print(LC_df.columns)

Index(['Location', 'region', 'super_region'], dtype='object')
Index(['country', 'year', 'sex', 'MR', 'region', 'super_region'], dtype='object')


In [16]:
for i in range(LC_df.shape[0]):
    for j in range(r_sr_df.shape[0]):
        if LC_df.iloc[i,LC_df.columns.get_loc("country")] == r_sr_df.iloc[j,r_sr_df.columns.get_loc("Location")]:
            LC_df.iloc[i,LC_df.columns.get_loc("region")] = r_sr_df.iloc[j,r_sr_df.columns.get_loc("region")]
            LC_df.iloc[i,LC_df.columns.get_loc("super_region")] = r_sr_df.iloc[j,r_sr_df.columns.get_loc("super_region")]

### Countries names Dcitionary

In [17]:
#"Write in a JSON
countries_dict = {
"Afghanistan":["Islamic Republic of Afghanistan ","Afghanistan"],
"Albania":["Republic of Albania","Albania"],
"Algeria":["People’s Democratic Republic of Algeria","Algeria"],
"American Samoa":["American Samoa"],
"Andorra":[" Principality of Andorra ","Andorra"],
"Angola":["Republic of Angola","Angola"],
"Antigua and Barbuda":["Antigua and Barbuda "],
"Argentina":["Republic of Argentina ","Argentina"],
"Armenia":["Republic of Armenia ","Armenia"],
"Australia":["Commonwealth of Australia ","Australia"],
"Austria":["Republic of Austria","Austria"],
"Azerbaijan":["Republic of Azerbaijan ","Azerbaijan"],
"Bahamas":["The Bahamas","Commonwealth of the Bahamas ", "Bahamas"],
"Bahrain":["Kingdom of Bahrain ","Bahrain"],
"Bangladesh":["People’s Republic of Bangladesh","Bangladesh"],
"Barbados":["Barbados"],
"Belarus":["Republic of Belarus","Belarus"],
"Belgium":["Kingdom of Belgium ","Belgium"],
"Belize":["Belize"],
"Benin":["Republic of Benin","Benin"],
"Bermuda":["Bermuda"],
"Bhutan":["Kingdom of Bhutan","Bhutan"],
"Bolivia":["Plurinational State of Bolivia (Plurinational State of) ","Bolivia"," Bolivia (Plurinational State of)"],
"Bosnia and Herzegovina":["Bosnia and Herzegovina","Bosnia and Herzegovina"],
"Botswana":["Republic of Botswana","Botswana"],
"Brazil":["Federative Republic of Brazil","Brazil"],
"Brunei":["Negara Brunei Darussalam","Brunei"," Brunei Darussalam"],
"Bulgaria":["Republic of Bulgaria","Bulgaria"],
"Burkina Faso":["Burkina Faso"],
"Burundi":["Republic of Burundi","Burundi"],
"Cabo Verde":["Republic of Cabo Verde","Cape Verde"," Cabo Verde"],
"Cambodia":["Kingdom of Cambodia","Cambodia"],
"Cameroon":["Republic of Cameroon","Cameroon"],
"Canada":["Canada"],
"Central African Republic":["Central African Republic"],
"Chad":["Republic of Chad","Chad"],
"Chile":["Republic of Chile","Chile"],
"China":["People’s Republic of China","China"],
"Colombia":["Republic of Colombia","Colombia"],
"Comoros":["Union of the Comoros","Comoros"],
"The Congo":["The Congo","Congo, Democratic Republic of the","Democratic Republic of the Congo","Congo (Democratic Republic of the)"],
"Congo, Republic of the":["Congo (Republic of the)", "Congo, Republic of the", "Republic of the Congo"],
"Costa Rica":["Republic of Costa Rica","Costa Rica"],
"Ivory coast":["Republic of Côte d’Ivoire","Cote d'Ivoire","Ivory coast","Côte d’Ivoire"],
"Croatia":["Republic of Croatia","Croatia"],
"Cuba":["Republic of Cuba","Cuba"],
"Cyprus":["Republic of Cyprus ","Cyprus"],
"Czech Republic":["Czech Republic"],
"Denmark":["Kingdom of Denmark","Denmark"],
"Djibouti":["Republic of Djibouti","Djibouti"],
"Dominica":["Commonwealth of Dominica","Dominica"],
"Dominican Republic":["Dominican Republic"],
"East Timor":["Democratic Republic of Timor-Leste","Timor Leste","Timor-Leste","East Timor"],
"Ecuador":["Republic of Ecuador","Ecuador"],
"Egypt":["Arab Republic of Egypt","Egypt"],
"El Salvador":["Republic of El Salvador","El Salvador"],
"Equatorial Guinea":["Republic of Equatorial Guinea","Equatorial Guinea"],
"Eritrea":["State of Eritrea","Eritrea"],
"Estonia":["Republic of Estonia","Estonia"],
"Ethiopia":["Federal Democratic Republic of Ethiopia ","Ethiopia"],
"Micronesia":["Federated States of Micronesia"," Micronesia (Federated States of)","Micronesia"],
"Fiji":["Republic of Fiji","Fiji"],
"Finland":["Republic of Finland","Finland"],
"France":["Republic of France","France"],
"Gabon":["Gabonese Republic (Gabon)","Gabon"],
"Gambia":["The Gambia","Republic of The Gambia", "Gambia"],
"Georgia":["Georgia"],
"Germany":["Federal Republic of Germany","Germany"],
"Ghana":["Republic of Ghana","Ghana"],
"Greece":["Hellenic Republic (Greece)","Greece"],
"Greenland":["Greenland"],
"Grenada":["Grenada"],
"Guam":["Guam"],
"Guatemala":["Republic of Guatemala","Guatemala"],
"Guinea":["Republic of Guinea","Guinea"],
"Guinea-Bissau":["Republic of Guinea-Bissau","Guinea-Bissau"],
"Guyana":["Republic of Guyana","Guyana"],
"Haiti":["Republic of Haiti","Haiti"],
"Honduras":["Republic of Honduras","Honduras"],
"Hong Kong":["Hong Kong","Hong Kong, China (SAR)"],
"Hungary":["Hungary"],
"Iceland":["Republic of Iceland","Iceland"],
"India":["Republic of India","India"],
"Indonesia":["Republic of Indonesia","Indonesia"],
"Iran":["Islamic Republic of Iran (Islamic Republic of)","Iran"," Iran (Islamic Republic of)"],
"Iraq":["Republic of Iraq","Iraq"],
"Ireland":["Ireland"],
"Israel":["State of Israel","Israel"],
"Italy":["Republic of Italy","Italy"],
"Jamaica":["Jamaica"],
"Japan":["Japan"],
"Jordan":["Hashemite Kingdom of Jordan","Jordan"],
"Kazakhstan":["Republic of Kazakhstan","Kazakhstan"],
"Kenya":["Republic of Kenya","Kenya"],
"Kiribati":["Republic of Kiribati","Kiribati"],
"North Korea":["North Korea", "Korea, North","Democratic People’s Republic of Korea","Korea (Democratic People's Republic of )"],
"South Korea":["Korea, South","Republic of Korea", "Korea (Republic of)", "South Korea"],
"Kosovo":["Kosovo"],
"Kuwait":["State of Kuwait","Kuwait"],
"Kyrgyzstan":["Kyrgyz Republic (Kyrgyzstan)","Kyrgyzstan"],
"Laos":["Lao People’s Democratic Republic","Laos"],
"Latvia":["Republic of Latvia","Latvia"],
"Lebanon":["Republic of Lebanon","Lebanon"],
"Lesotho":["Kingdom of Lesotho","Lesotho"],
"Liberia":["Republic of Liberia","Liberia"],
"Libya":["State of Libya","Libya"],
"Liechtenstein":["Principality of Liechtenstein"," Liechtenstein"],
"Lithuania":["Republic of Lithuania"," Lithuania"],
"Luxembourg":["Grand Duchy of Luxembourg","Luxembourg"],
"Macedonia":["Macedonia","The former Yugoslav Republic of Macedonia ","Macedonia (The former Yugoslav Republic of) "],
"Madagascar":["Republic of Madagascar","Madagascar"],
"Malawi":["Republic of Malawi","Malawi"],
"Malaysia":["Malaysia"],
"Maldives":["Republic of Maldives","Maldives"],
"Mali":["Republic of Mali","Mali"],
"Malta":["Republic of Malta","Malta"],
"Marshall Islands":["Republic of the Marshall Islands","Marshall Islands"],
"Mauritania":["Islamic Republic of Mauritania","Mauritania"],
"Mauritius":["Republic of Mauritius","Mauritius"],
"Mexico":["United Mexican States (Mexico)","Mexico"],
"Micronesia, Federated States of":["Micronesia, Federated States of","Federated States of Micronesia (Federated States of)", "Micronesia (Federated States of)"],
"Moldova":["Republic of Moldova","Moldova"," Moldova (Republic of)"],
"Monaco":["Principality of Monaco", "Monaco"],
"Mongolia":["Mongolia"],
"Montenegro":["Montenegro"],
"Morocco":["Kingdom of Morocco","Morocco"],
"Mozambique":["Republic of Mozambique","Mozambique"],
"Myanmar":["Republic of the Union of Myanmar","Myanmar (Burma)"," Myanmar"],
"Namibia":["Republic of Namibia","Namibia"],
"Nauru":["Republic of Nauru"],
"Nepal":["Federal Democratic Republic of Nepal","Nepal"],
"Netherlands":["Kingdom of the Netherlands","Netherlands"],
"New Zealand":["New Zealand"],
"Nicaragua":["Republic of Nicaragua","Nicaragua"],
"Niger":["Republic of the Niger","Niger"],
"Nigeria":["Federal Republic of Nigeria","Nigeria"],
"Northern Mariana Islands":["Northern Mariana Islands"],
"Norway":["Kingdom of Norway","Norway"],
"Oman":["Sultanate of Oman","Oman"],
"Pakistan":["Islamic Republic of Pakistan","Pakistan"],
"Palestine":["State of Palestine"," Palestine, State of","Palestine"],
"Palau":["Republic of Palau", " Palau"],
"Panama":["Republic of Panama","Panama"],
"Papua New Guinea":["Independent State of Papua New Guinea","Papua New Guinea"],
"Paraguay":["Republic of Paraguay","Paraguay"],
"Peru":["Republic of Peru","Peru"],
"Philippines":["Republic of the Philippines","Philippines"],
"Poland":["Republic of Poland","Poland"],
"Portugal":["Portuguese Republic (Portugal) ","Portugal"],
"Puerto Rico":["Puerto Rico"],
"Qatar":["State of Qatar","Qatar"],
"Romania":["Romania"],
"Russia":["Russian Federation","Russia"],
"Rwanda":["Republic of Rwanda","Rwanda"],
"Saint Kitts and Nevis":["Saint Kitts and Nevis"],
"Saint Lucia":["Saint Lucia"],
"Saint Vincent and the Grenadines":["Saint Vincent and the Grenadines"],
"Samoa":["Independent State of Samoa","Samoa"],
"San Marino":["Republic of San Marino", "San Marino"],
"Sao Tome and Principe":["Democratic Republic of Sao Tome and Principe ","Sao Tome and Principe"],
"Saudi Arabia":["Kingdom of Saudi Arabia","Saudi Arabia"],
"Senegal":["Republic of Senegal","Senegal"],
"Serbia":["Republic of Serbia","Serbia"],
"Seychelles":["Republic of Seychelles","Seychelles"],
"Sierra Leone":["Republic of Sierra Leone","Sierra Leone"],
"Singapore":["Republic of Singapore","Singapore"],
"Slovakia":["Slovak Republic (Slovakia)","Slovakia"],
"Slovenia":["Republic of Slovenia","Slovenia"],
"Solomon Islands":["Solomon Islands"],
"Somalia":["Federal Republic of Somalia","Somalia"],
"South Africa":["Republic of South Africa","South Africa"],
"Spain":["Kingdom of Spain","Spain"],
"Sri Lanka":["Democratic Socialist Republic of Sri Lanka ","Sri Lanka"],
"Sudan":["Republic of the Sudan","Sudan"],
"South Sudan":["Republic of South Sudan","Sudan, South","South Sudan"],
"Suriname":["Republic of Suriname","Suriname"],
"Swaziland":["Kingdom of Swaziland","Swaziland"],
"Sweden":["Kingdom of Sweden","Sweden"],
"Switzerland":["Swiss Confederation (Switzerland)","Switzerland"],
"Syria":["Syrian Arab Republic","Syria"],
"Taiwan":["Taiwan"],
"Tajikistan":["Republic of Tajikistan","Tajikistan"],
"Tanzania":["United Republic of Tanzania","Tanzania"," Tanzania (United Republic of)"],
"Thailand":["Kingdom of Thailand","Thailand"],
"Togo":["Republic of Togo","Togo"],
"Tonga":["Kingdom of Tonga","Tonga"],
"Trinidad and Tobago":["Republic of Trinidad and Tobago","Trinidad and Tobago"],
"Tunisia":["Republic of Tunisia","Tunisia"],
"Turkey":["Republic of Turkey","Turkey"],
"Turkmenistan":["Turkmenistan","Turkmenistan"],
"Tuvalu":["Tuvalu"],
"Uganda":["Republic of Uganda", " Uganda"],
"Ukraine":["Ukraine"],
"United Arab Emirates":["United Arab Emirates"],
"United Kingdom":["United Kingdom of Great Britain and Northern Ireland","United Kingdom"],
"United States":["United States of America","United States"],
"Uruguay":["Oriental Republic of Uruguay ","Uruguay"],
"Uzbekistan":["Republic of Uzbekistan","Uzbekistan"],
"Vanuatu":["Republic of Vanuatu","Vanuatu"],
"Vatican City":[ "Vatican City"],
"Venezuela":["Bolivarian Republic of Venezuela "," Venezuela (Bolivarian Republic of)","Venezuela"],
"Vietnam":["Socialist Republic of Viet Nam ","Vietnam"," Viet Nam"],
"Virgin Islands, U.S.":["Virgin Islands, U.S."],
"Yemen":["Republic of Yemen ","Yemen"],
"Zambia":["Republic of Zambia ","Zambia"],
"Zimbabwe":["Republic of Zimbabwe ","Zimbabwe"]
}

### Combine LC_df and HDI_df_long to include:
> ### country, id, region, super-region, year, sex, MR, HDI 

In [18]:
#LC_HDI ('country', 'id','region', 'super_region', 'year', 'sex', 'MR', 'HDI')
print(LC_df.columns)
print(HDI_df_long.columns)

Index(['country', 'year', 'sex', 'MR', 'region', 'super_region'], dtype='object')
Index(['country', 'id', 'year', 'HDI'], dtype='object')


In [19]:
#Number of unique countries in each dataframe
ori_LC_df = LC_df.copy()
ori_HDI_df_long = HDI_df_long.copy()
print(len(ori_LC_df.country.unique()))
print(len(ori_HDI_df_long.country.unique()))


195
188


#### First unify countries names in both dataframes

In [20]:
HDI_df_long.country.unique()

array([' Afghanistan', ' Albania', ' Algeria', ' Andorra', ' Angola',
       ' Antigua and Barbuda', ' Argentina', ' Armenia', ' Australia',
       ' Austria', ' Azerbaijan', ' Bahamas', ' Bahrain', ' Bangladesh',
       ' Barbados', ' Belarus', ' Belgium', ' Belize', ' Benin', ' Bhutan',
       ' Bolivia (Plurinational State of)', ' Bosnia and Herzegovina',
       ' Botswana', ' Brazil', ' Brunei Darussalam', ' Bulgaria',
       ' Burkina Faso', ' Burundi', ' Cabo Verde', ' Cambodia',
       ' Cameroon', ' Canada', ' Central African Republic', ' Chad',
       ' Chile', ' China', ' Colombia', ' Comoros', ' Congo',
       ' Congo (Democratic Republic of the)', ' Costa Rica', ' Croatia',
       ' Cuba', ' Cyprus', ' Czech Republic', ' Denmark', ' Djibouti',
       ' Dominica', ' Dominican Republic', ' Ecuador', ' Egypt',
       ' El Salvador', ' Equatorial Guinea', ' Eritrea', ' Estonia',
       ' Ethiopia', ' Fiji', ' Finland', ' France', ' Gabon', ' Gambia',
       ' Georgia', ' German

In [21]:
#TO OPTIMISE : Move the normalisation to time of reading
def country_unification(country_name , country_dict):
    name = ''
    for item in country_dict.items():
        if [s for s in item[1] if country_name.strip().replace("’", "'") == s.strip().replace("’", "'")]:
            name = item[0].strip()
            #print(name)
    return name

In [22]:
#Check for duplicates in HDI-df_long
duplicates_HDI = set()
for i in HDI_df_long.country.unique():
    unified_name = country_unification(i, countries_dict)
    if unified_name in duplicates_HDI:
        print('it is already there: {} -> {}'.format(i, unified_name))
    HDI_df_long.country[HDI_df_long.country==i] = unified_name
    duplicates_HDI.add(unified_name)

In [23]:
#Check for duplicates in LC_df
duplicates_LC = set()
for i in LC_df.country.unique():
    unified_name = country_unification(i, countries_dict)
    if unified_name in duplicates_LC:
        print('it is already there: {} -> {}'.format(i, unified_name))
    LC_df.country[LC_df.country == i] = unified_name
    duplicates_LC.add(unified_name)

In [24]:
#Make sure all countries are normalised
print (len(LC_df.country.unique()))
print(len(HDI_df_long.country.unique()))


195
188


In [25]:
print(len(duplicates_HDI))
print(len(duplicates_LC))

188
195


#### Control the missing countries

In [26]:
def country_missing(ori_list, res_list):
    Countries = [x for x in ori_list if x not in res_list]
    return Countries

In [27]:
LC_df_missing = country_missing(ori_LC_df.country.unique(), LC_df.country.unique())

In [28]:
len(LC_df_missing)

8

In [29]:
HDI_df_long_missing = country_missing(ori_HDI_df_long.country.unique(), HDI_df_long.country.unique())

In [30]:
len(HDI_df_long_missing)

188

In [31]:
#HDI missing countries      HDI_df_long.country.unique()-Countries
HDI_df_list = [x.strip() for x in HDI_df_long.country.unique()]
Countries_list = [x.strip() for x in HDI_df_long_missing]
l3 = [x for x in HDI_df_list if x not in Countries_list]

In [32]:
print(len(HDI_df_long_missing))
print(len(l3))


188
19


In [33]:
#LC missing countries      HDI_df_long.country.unique()-Countries
LC_df_list = [x.strip() for x in LC_df.country.unique()]
Countries_list = [x.strip() for x in LC_df_missing]
l3 = [x for x in LC_df_list if x not in Countries_list]

In [34]:
print(len(LC_df_missing))
print(len(l3))


8
195


#### Creat LC_HDI : combine LC_df and HDI_df_long 

In [35]:
LC_df.head()

,country,year,sex,MR,region,super_region
0,Afghanistan,1990,Male,31.086848,5,2
1,Afghanistan,1990,Female,11.711023,5,2
2,Afghanistan,1990,Both,21.094622,5,2
3,Afghanistan,1991,Male,31.152613,5,2
4,Afghanistan,1991,Female,11.667114,5,2


In [36]:
HDI_df_long.head()

,country,id,year,HDI
0,Afghanistan,0,1990,0.295
1,Afghanistan,0,1991,0.300
2,Afghanistan,0,1992,0.309
3,Afghanistan,0,1993,0.305
4,Afghanistan,0,1994,0.300


In [37]:
col_names = ["id" , "country", "year", "sex", "MR", "region", "super_region", "HDI"]
LC_HDI = pd.DataFrame(columns= col_names)

In [39]:
for i in range(LC_df.shape[0]):
    for j in range(HDI_df_long.shape[0]):
        country_HDI = HDI_df_long.iloc[j , HDI_df_long.columns.get_loc("country")].strip()
        country_LC = LC_df.iloc[i,LC_df.columns.get_loc("country")].strip()
        if country_HDI== country_LC:
            year_HDI = str(HDI_df_long.iloc[j,HDI_df_long.columns.get_loc("year")])
            year_LC = str(LC_df.iloc[i,LC_df.columns.get_loc("year")])
            if year_HDI == year_LC:
                LC_HDI = LC_HDI.append({'id': HDI_df_long.iloc[j,HDI_df_long.columns.get_loc("id")],
                                        'country': HDI_df_long.iloc[j,HDI_df_long.columns.get_loc("country")],
                                        'year': LC_df.iloc[i,LC_df.columns.get_loc("year")],
                                        'sex': LC_df.iloc[i,LC_df.columns.get_loc("sex")],
                                        'MR': LC_df.iloc[i,LC_df.columns.get_loc("MR")],
                                        'region': LC_df.iloc[i,LC_df.columns.get_loc("region")],
                                        'super_region': LC_df.iloc[i,LC_df.columns.get_loc("super_region")],
                                        'HDI': HDI_df_long.iloc[j,HDI_df_long.columns.get_loc("HDI")]
                                       }, ignore_index=True)

In [40]:
LC_HDI.head()

,id,country,year,sex,MR,region,super_region,HDI
0,0,Afghanistan,1990,Male,31.086848,5,2,0.295
1,0,Afghanistan,1990,Female,11.711023,5,2,0.295
2,0,Afghanistan,1990,Both,21.094622,5,2,0.295
3,0,Afghanistan,1991,Male,31.152613,5,2,0.300
4,0,Afghanistan,1991,Female,11.667114,5,2,0.300


In [41]:
LC_HDI.to_csv("Data/LC_HDI.csv")

> ### Separate Both, Female and Male

In [42]:
LC_HDI_both = LC_HDI[LC_HDI.sex=="Both"]
LC_HDI_both.to_csv("Data/LC_HDI_both.csv")

In [43]:
LC_HDI_female = LC_HDI[LC_HDI.sex=="Female"]
LC_HDI_female.to_csv("Data/LC_HDI_female.csv")

In [44]:
LC_HDI_male = LC_HDI[LC_HDI.sex=="Male"]
LC_HDI_male.to_csv("Data/LC_HDI_male.csv")